# Prerequisites
본 `ipynb` 은 `Python=3.12` 에서 작성하였습니다. Package dependency 를 해결하기 위해 아래 cell 을 실행해주세요.

## Install Python packages

In [ ]:
%pip -q install -U azure-identity azure-search-documents azure-ai-documentintelligence langchain langchain-community langchain-openai

## Load environment variables from a .env file
secret 노출을 피하고 notebook 들간의 일관된 환경변수를 설정하기 위해 `dotenv` 을 이용한다.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_WESTUS3_ENDPOINT = os.getenv("AZURE_OPENAI_WESTUS3_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_MS_FOUNDRY_API_KEY")
AZURE_OPENAI_EMBEDDING_DEPLOYMENT = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")
AZURE_AI_SEARCH_ENDPOINT = os.getenv("AZURE_AI_SEARCH_ENDPOINT")
AZURE_AI_SEARCH_ADMIN_KEY = os.getenv("AZURE_AI_SEARCH_ADMIN_KEY")
AZURE_DOCUMENTINTELLIGENCE_ENDPOINT = os.getenv("AZURE_DOCUMENTINTELLIGENCE_ENDPOINT")
AZURE_DOCUMENTINTELLIGENCE_API_KEY = os.getenv("AZURE_DOCUMENTINTELLIGENCE_API_KEY")

# AI Enrichment
Azure 의 Document Intelligence 는 비정형 문서에서 텍스트, 테이블, 폼 등을 추출하여 구조화하는데 필수적인 AI 서비스이다. PDF 등과 같은 문서를 ETL pipeline 을 태울 때 유용하게 사용된다.

## Loading Document
Native 하게 Azure SDK 를 써도 되지만, langchain 이 abstraction 을 잘해놓아서 이를 활용해보자.

In [ ]:
from langchain_community.document_loaders.doc_intelligence import AzureAIDocumentIntelligenceLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores.azuresearch import AzureSearch

loader = AzureAIDocumentIntelligenceLoader(
    api_endpoint=AZURE_DOCUMENTINTELLIGENCE_ENDPOINT,
    api_key=AZURE_DOCUMENTINTELLIGENCE_API_KEY,
    file_path="./resources/KB주택시장리뷰_2025년 10월호.pdf",
    api_model="prebuilt-layout",
)
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunks = splitter.split_documents(docs)


In [ ]:
# metadata 에 있는 값들을 document 에 들어가기에 중복되는 key 들을 제거한다.
for d in chunks:
    d.metadata.pop("content", None)          # 충돌 키 제거
    d.metadata.pop("content_vector", None)   # (안전)
    d.metadata.pop("id", None)

## Upload Documents
위에서 parsing 한 chunks 들을 embeddings 해서 AI Search 로 업로드하자.

In [ ]:
index_name = "housing"

emb = AzureOpenAIEmbeddings(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    azure_deployment=AZURE_OPENAI_EMBEDDING_DEPLOYMENT,
    openai_api_version=AZURE_OPENAI_API_VERSION,
)

# 여기선 index name 이 없을 때 schema 를 추론하여 자동 생성해준다.
vs = AzureSearch(
    azure_search_endpoint=AZURE_AI_SEARCH_ENDPOINT,
    azure_search_key=AZURE_AI_SEARCH_ADMIN_KEY,
    index_name=index_name,
    embedding_function=emb.embed_query,
)
vs.add_documents(documents=chunks)

## Process search query
한번 search methods 를 사용해보자.

In [ ]:
# Perform a similarity search
docs = vs.similarity_search(
    query="매매가격 상승폭이 가장 컸던 지역은?",
    k=3,
    search_type="hybrid",
)
for idx, doc in enumerate(docs):
    print(f"\n=== Document {idx + 1} ===")
    print(doc.page_content)

## Delete the index
사용하지 않는다면, index 는 삭제하자.

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient

index_client = SearchIndexClient(
    endpoint=AZURE_AI_SEARCH_ENDPOINT,
    credential=AzureKeyCredential(AZURE_AI_SEARCH_ADMIN_KEY),
).delete_index(index_name)